In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.chains.openai_functions import create_structured_output_runnable
from enum import Enum
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict, Union, Literal
from langchain_core.messages import BaseMessage, HumanMessage
import operator
from langchain.pydantic_v1 import BaseModel

# class Next(Enum):
#     ANALYSIS = 'spatial_analysis_worker'
#     MAP = 'client_map_worker'


class Worker(Enum):
    CODE = 'code_worker'
    SUMMARY = 'summary_worker'

    @classmethod
    def get_description(cls, worker: 'Worker'):    
        return {
            Worker.CODE.value: 'A coding worker/agent that can produce and execute Python code',
            Worker.SUMMARY.value: 'A summary writing worker/agent that can produce summaries of human-AI conversations'
        }.get(worker.value)

class AgentState(TypedDict):
    initial_query: str
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: Worker


def create_agent_supervisor(members: Sequence[Worker]):
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        " following workers:\n{members}\n\n Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    options = ["FINISH"] + [m.value for m in members]
    print(options)

    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }

    bullet_point_list = "\n".join(f"{i+1}) {member.value} - {Worker.get_description(member)}"
                                  for i, member in enumerate(members))

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                ( 
                    "Given the conversation above, who should act next, if any?"
                    ' Return "FINISH" if the initial human query ("{initial_query}") has been answered?\n\n'
                    "Select one of: {options}\n\n"
                    "Give you answer as a dict: {'next': <selected_option>}"
                )
            )
        ]
    ).partial(options=options, members=bullet_point_list)

    llm = ChatOpenAI(model="gpt-3.5-turbo-0125", streaming=True)

    # return create_structured_output_runnable(
    #     {
    #         'code_worker': 'string',
    #         'summary_worker': 'string',
    #         'FINISH': 'string',
    #     }, 
    #     llm, 
    #     prompt
    # )
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        # | llm
        | JsonOutputFunctionsParser()
    )

KeyboardInterrupt: 

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import HumanMessage
from langchain.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


def create_agent(llm: ChatOpenAI, tools: Sequence[BaseTool], system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    if len(tools) == 0:
        return (
            ChatPromptTemplate.from_template(system_prompt)
            | llm
            | StrOutputParser()
        )

    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


async def agent_node(state: AgentState, agent, name):
    result = await agent.ainvoke(state)
    # content = result["output"] if isinstance(result, dict) else result
    return {"messages": [HumanMessage(content=result['output'], name=name)]}

In [ ]:
import functools
from langgraph.graph import StateGraph, END
from langchain_experimental.tools import PythonREPLTool

supervisor_chain = create_agent_supervisor([e for e in Worker])

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", streaming=True)

summary_agent = create_agent(llm, [], "You are a summary agent. Write a summary of the conversation so far: {messages}")
summary_node = functools.partial(
    agent_node, agent=summary_agent, name="Summarizer")

code_agent = create_agent(
    llm,
    [PythonREPLTool()],
    "You are a coding agent.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

SUPERVISOR = 'supervisor'

workflow = StateGraph(AgentState)
workflow.add_node(Worker.SUMMARY.value, summary_node)
workflow.add_node(Worker.CODE.value, code_node)
workflow.add_node(SUPERVISOR, supervisor_chain)

In [ ]:
for member in Worker:
    workflow.add_edge(member.value, SUPERVISOR)

conditional_map = {k.value: k.value for k in Worker}
conditional_map["FINISH"] = END
workflow.add_conditional_edges(
    SUPERVISOR, lambda x: x["next"], conditional_map)

    
workflow.set_entry_point(SUPERVISOR)

app = workflow.compile()

In [ ]:
messages = [HumanMessage(content="What is 576 * 5 / 13?")]
async for event in app.astream_events(
    {
        'initial_query': messages[0].content,
        "messages": messages
    },
    {"recursion_limit": 100},
    version="v1"
):
    kind = event["event"]
    print(event)
    # if kind == "on_chat_model_stream":
    #     content = event["data"]["chunk"].content
    #     if content:
    #         # Empty content in the context of OpenAI means
    #         # that the model is asking for a tool to be invoked.
    #         # So we only print non-empty content
    #         print(content, end="|")
    # elif kind == "on_tool_start":
    #     print("--")
    #     print(
    #         f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
    #     )
    # elif kind == "on_tool_end":
    #     print(f"Done tool: {event['name']}")
    #     print(f"Tool output was: {event['data'].get('output')}")
    #     print("--")

NotImplementedError in LogStreamCallbackHandler.on_chain_end callback: NotImplementedError("Trying to load an object that doesn't implement serialization: {'lc': 1, 'type': 'not_implemented', 'id': ['builtins', 'object'], 'repr': '<object object at 0x7f28d52265e0>'}")


{'event': 'on_chain_start', 'run_id': '45c41d8c-73a6-4133-817e-bfedc58d7c1e', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'initial_query': 'What is 576 * 5 / 13?', 'messages': [HumanMessage(content='What is 576 * 5 / 13?')]}}}
{'event': 'on_chain_start', 'name': '__start__', 'run_id': '4b773fa8-4a39-436e-a911-0bcccb226c59', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {}, 'data': {}}
{'event': 'on_chain_stream', 'name': '__start__', 'run_id': '4b773fa8-4a39-436e-a911-0bcccb226c59', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {}, 'data': {'chunk': {'initial_query': 'What is 576 * 5 / 13?', 'messages': [HumanMessage(content='What is 576 * 5 / 13?')]}}}
{'event': 'on_chain_end', 'name': '__start__', 'run_id': '4b773fa8-4a39-436e-a911-0bcccb226c59', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {}, 'data': {'input': {'initial_query': 'What is 576 * 5 / 13?', 'messages': [HumanMessage(content='What is 576 * 5 / 13?')]}, 'output'

KeyError: 'Input to ChatPromptTemplate is missing variables {"\'next\'"}.  Expected: ["\'next\'", \'initial_query\', \'messages\'] Received: [\'initial_query\', \'messages\', \'next\']'

In [ ]:
# initial_msg = "What is 576 * 5 / 13?"

# async for s in app.astream(
#     {
#         'initial_query': initial_msg,
#         "messages": [HumanMessage(
#         content=initial_msg)]},
#     {"recursion_limit": 100},
# ):
#     print(s)
#     # if "__end__" not in s:
#     #     print(s)
#     #     print("----")